In [1]:
# load an article
import os
ABSOLUTE_PATH = os.getcwd()

import sys
ROOT_PATH = '/'.join(ABSOLUTE_PATH.split('/')[:-1])
sys.path.append(ROOT_PATH)


from main.document import Document
from pprint import pprint
input_file_path = f'{ROOT_PATH}/data/article.txt'
document = Document.load_from_local(input_file_path)
pprint(document.content)

('Teenagers in foster care in Scotland are being moved too often, according to '
 'a campaign group. Research carried out\n'
 'by the Fostering Network suggests almost half of fostered young people are '
 'already living with their third foster\n'
 'family since going into care. The group has warned that 750 more foster '
 'carers are \\"urgently\\" needed to meet the\n'
 'demands of the care system. It urged people to \\"open their hearts and '
 'homes\\" to vulnerable youngsters. Currently,\n'
 'more than 5,500 children are in foster care in Scotland, living with 4,400 '
 'families and carers. The Fostering Network\n'
 'surveyed 250 children, teenagers and foster carers across Scotland and '
 'discovered that many young people had failed\n'
 'to find stability. Almost half were already living with their third family, '
 'a quarter were with their fourth family\n'
 'and about 20 were living with their 10th family since going into care. There '
 'was a particular need for homes to be\n

In [2]:
# Customized summarizer
has_title = True                   # has a title
min_num_of_char_in_title = 32      # 32 <= title length <= 80 
max_num_of_char_in_title = 80
compression_rate = 0.3             # compression rate = 0.3
min_num_of_paragraph = 2           # 2 <= number of paragraphs <= 4
max_num_of_paragraph = 4

from openai import AsyncOpenAI
llm_api_client = AsyncOpenAI(base_url="http://localhost:11434/v1", api_key="dummy_key")
model = "deepseek-r1:8b"

from main.summarizer import BestHitLLMSummarizer
summarizer = BestHitLLMSummarizer(
    client=llm_api_client, 
    model=model,
    has_title=has_title,
    min_num_of_char_in_title=min_num_of_char_in_title,
    max_num_of_char_in_title=max_num_of_char_in_title,
    min_num_of_paragraph=min_num_of_paragraph,
    max_num_of_paragraph=max_num_of_paragraph,
    num_tries=5,
    llm_as_judge=False)


report = await summarizer.summarize(document=document)
pprint(report)

/Users/yihong/Documents/ai-summarize-document/venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Summarize at iteration 0
Summarize at iteration 1
Summarize at iteration 2
Summarize at iteration 3
Summarize at iteration 4
Report(title='Foster care instability crisis in Scotland highlighted by campaign group study\n\n', content="According to the Fostering Network's research, teenagers in Scotland's foster care system experience excessive movement between families. Almost half of fostered youth have lived with three different foster families, while a quarter are with their fourth, and some have as many as ten placements, indicating a severe lack of stability in many cases.\n\n\n\n\n\nThis instability significantly hinders education and wellbeing, whereas stability leads to better outcomes, as demonstrated by personal testimonies like Carla's, who credits the Randalls with transforming her life.\n\n\n\n\n\n\nThe Fostering Network urgently calls for 750 additional foster carers to reduce placements and emphasize the need for homes that offer love and security, especially for vulnerabl

In [6]:
# Evaluate the report
# 1) metric for checking if the report has title
from main.metrics import HasTitleMetricExtractor
has_title_metric = HasTitleMetricExtractor().extract(report=report)
print(f'metric name: {has_title_metric.name}, metric value: {has_title_metric.value}')

metric name: Has-Title-Metric, metric value: 1


In [10]:
# 2) metric for measuring the number of characters in the title
from main.metrics import TitleLengthMetricExtractor
title_length_metric = TitleLengthMetricExtractor().extract(report=report)
print(f'metric name: {title_length_metric.name}, metric value: {title_length_metric.value}')

metric name: Number-Of-Chars-In-Title-Metric, metric value: 78


In [11]:
# 3) metric for computing the number of paragraphs 
from main.metrics import NumberOfParagraphMetricExtractor
num_of_para_metric = NumberOfParagraphMetricExtractor().extract(report=report)
print(f'metric name: {num_of_para_metric.name}, metric value: {num_of_para_metric.value}')

metric name: Number-Of-Paragraphs-Metric, metric value: 11


In [14]:
# 4) metric for computing the number of tokens
from main.metrics import NumberOfTokenMetricExtractor
num_of_token_metric = NumberOfTokenMetricExtractor().extract(report=report)
compression_rate = 1.0 * float(num_of_token_metric.value) / len([w for w in document.content.split() if w.strip()])
print(f'metric name: {num_of_token_metric.name}, metric value: {num_of_token_metric.value}, compression rate: {compression_rate}')

metric name: Number-Of-Tokens-Metric, metric value: 134, compression rate: 0.27346938775510204


In [16]:
# 5) metric for computing bert-score
from main.llm_as_judge import Reference
from main.metrics import BertScoreMetricExtractor
bert_score_metric = BertScoreMetricExtractor(reference=Reference(content=document.content)).extract(report=report)
print(f'metric name: {bert_score_metric.name}, metric value: {bert_score_metric.value}')

metric name: Bert-Score-Metric, metric value: 0.6185052990913391


In [17]:
# 6) metric for computing Rouge-score
from main.metrics import RougeScoreMetricExtractor
rouge_score_metric = RougeScoreMetricExtractor(reference=Reference(content=document.content)).extract(report=report)
print(f'metric name: {rouge_score_metric.name}, metric value: {rouge_score_metric.value}')

metric name: Rouge-Score-Metric, metric value: 0.35766423357664234


In [19]:
# 7) LLM-as-judge for judging the correctness
from main.metrics import CorrectnessMetricExtractor
correctness_metric = await CorrectnessMetricExtractor(client=llm_api_client, 
                                                model=model, 
                                                reference=Reference(content=document.content)).extract(report=report)
print(f'metric name: {correctness_metric.name}, metric value: {correctness_metric.value}')

metric name: Correctness-Metric, metric value: 1


In [20]:
# 8) LLM-as-judge for judging the completeness
from main.metrics import CompletenessMetricExtractor
completeness_metric = await CompletenessMetricExtractor(client=llm_api_client, 
                                                        model=model, 
                                                        reference=Reference(content=document.content)).extract(report=report)
print(f'metric name: {completeness_metric.name}, metric value: {completeness_metric.value}')

metric name: Completeness-Metric, metric value: 1
